In [ ]:
#!pip install tensorflow==1.15.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, SpatialDropout1D, Concatenate, Dropout, RepeatVector, Permute, Multiply, Lambda, TimeDistributed, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence, text
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
MAX_SENTENCES = 15
max_len = 100
NB_CLASSES = 3

In [50]:
# total_data =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/israel/dataset/labeled_data.csv')
# train, test = train_test_split(total_data, test_size = 0.25, random_state = 42)
# print(train.head())
# print('훈련용 리뷰의 개수 :', len(train))
# print(test.head())
# print('테스트용 리뷰의 개수 :', len(test))
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/israel/dataset/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/israel/dataset/test.csv")
print(train.head())
print('훈련용 리뷰의 개수 :', len(train))
print(test.head())
print('테스트용 리뷰의 개수 :', len(test))

       textID  ... label
0  6e0c6d75b1  ...     1
1  fc2cbefa9d  ...     1
2  16fab9f95b  ...     1
3  e48b0b8a23  ...     1
4  e00c6ef376  ...     1

[5 rows x 5 columns]
훈련용 리뷰의 개수 : 16363
       textID  ... label
0  96d74cb729  ...     1
1  01082688c6  ...     1
2  33987a8ee5  ...     1
3  726e501993  ...     1
4  33f19050cf  ...     1

[5 rows x 4 columns]
테스트용 리뷰의 개수 : 2104


In [52]:
# NLTK의 불용어
stop_words = set(stopwords.words('english'))
print('불용어 개수 :', len(stop_words))
print(stop_words)

불용어 개수 : 179
{'weren', 'or', 'both', 'into', 'shouldn', 'when', 'other', 'in', "you've", 'me', 'of', 'needn', 'during', 'down', "that'll", "weren't", 'll', 'the', "wasn't", 'about', 'between', 've', 'were', 'itself', 'is', 'and', 'a', 'whom', 'am', 'won', 'because', 'until', 'out', 'theirs', 'only', 'on', 'wasn', 'i', 'it', 'have', 'again', 'at', 'he', 'if', 'ma', 'these', 'them', 'those', 'ours', 'she', 'this', 'there', 'while', 'having', 'yourselves', 'each', 'does', "doesn't", 'y', "you're", "should've", 'been', 'do', "shan't", 'why', "needn't", 'from', 'your', 'not', 'we', 'more', "she's", "couldn't", 'should', 'to', 'before', 'ourselves', 'then', "isn't", 'themselves', 'just', 'by', 'that', "you'll", 'for', 'up', 'him', 'are', 'did', 'aren', 'all', 'some', 'their', 'didn', 'can', 'mightn', "hadn't", 'myself', 'so', "won't", 'too', 'himself', 't', 'below', 'here', 'most', 'doing', 'don', 'will', 'was', 'has', 'you', 'through', 're', 'over', 'nor', "haven't", 'his', 'as', 'off', "di

In [53]:
# 전처리 함수 내 사용
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [54]:
# 전처리 함수
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower() # 텍스트 소문자화
    # sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","",sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah
    return sentence

In [55]:
train['text'] = train['text'].apply(preprocess_sentence)
test['text'] = test['text'].apply(preprocess_sentence)

In [56]:
x_train = train['text'].tolist()
y_train  = train['label'].tolist()

In [57]:
#@title
# 토큰의 인덱스 생성
tokenizer = text.Tokenizer(num_words=None)
tokenizer.fit_on_texts(x_train)

In [58]:
X_train = np.zeros((len(train), MAX_SENTENCES, max_len), dtype='int32')

for i, sentences in enumerate(x_train):
    token = [word for word in sentences.split()]
    tokenized_sentences = tokenizer.texts_to_sequences(token)
    tokenized_sentences = sequence.pad_sequences(tokenized_sentences, maxlen=max_len)

    pad_size = MAX_SENTENCES - tokenized_sentences.shape[0]

    if pad_size <= 0:  # tokenized_sentences.shape[0] < MAX_SEQUENCES
        tokenized_sentences = tokenized_sentences[:MAX_SENTENCES]
    else:
        tokenized_sentences = np.pad(
            tokenized_sentences, ((0, pad_size), (0, 0)),
            mode='constant', constant_values=0
        )
    
    X_train[i] = tokenized_sentences[None, ...]


In [59]:
X_train.shape

(16363, 15, 100)

In [60]:
Y_train = to_categorical(y_train, NB_CLASSES)

In [61]:

embeddings_index = {}
# f = open('/content/drive/MyDrive/Colab Notebooks/israel/glove/glove.6B.100d.txt','r',encoding='utf-8')
f = open('/content/drive/MyDrive/Colab Notebooks/israel/glove/glove.6B.100d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:12, 32957.83it/s]

Found 400000 word vectors.


In [62]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 17654/17654 [00:00<00:00, 481072.79it/s]


In [63]:
embedding_matrix.shape

(17655, 100)

In [64]:
class AttentionLayer(tf.keras.Model):
    def __init__(self, attention_dim, **kwargs):
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='Attention_Weight',
                                 shape=(input_shape[-1], self.attention_dim),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(name='Attention_Bias',
                                 shape=(self.attention_dim, ),
                                 initializer='random_normal',
                                 trainable=True)
        self.u = self.add_weight(name='Attention_Context_Vector',
                                 shape=(self.attention_dim, 1),
                                 initializer='random_normal',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        # refer to the original paper
        # link: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
        u_it = K.tanh(K.dot(x, self.W) + self.b)
        a_it = K.dot(u_it, self.u)
        a_it = K.squeeze(a_it, -1)
        a_it = K.softmax(a_it)
        
        return a_it
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])



In [65]:
lstm_dim = 64


sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], input_length=max_len, mask_zero = True)(sequence_input)
bilstm_sentence = Bidirectional(LSTM(lstm_dim, dropout=0.5, return_sequences = True))(embedded_sequences)
word_attention = AttentionLayer(100)(bilstm_sentence)
repeated_word_attention = RepeatVector(lstm_dim * 2)(word_attention)
repeated_word_attention = Permute([2, 1])(repeated_word_attention)
sentence_representation = Multiply()([bilstm_sentence, repeated_word_attention])
sentence_representation = Lambda(lambda x: K.sum(x, axis=1))(sentence_representation)
sentence_encoder = Model(inputs=[sequence_input], 
                         outputs=[sentence_representation])

# then, build a document encoder
document_input = Input(shape=(MAX_SENTENCES,100), dtype='int32')
embedded_document = TimeDistributed(sentence_encoder)(document_input)
bilstm_document = Bidirectional(LSTM(lstm_dim, return_sequences=True))(embedded_document)

# sentence attention computation
sentence_attention = AttentionLayer(100)(bilstm_document)

# sentence attention application
repeated_sentence_attention = RepeatVector(lstm_dim * 2)(sentence_attention)
repeated_sentence_attention = Permute([2, 1])(repeated_sentence_attention)

# compute document representation as the weighted sum of sentence representations
document_representation = Multiply()([bilstm_document, repeated_sentence_attention])
document_representation = Lambda(lambda x: K.sum(x, axis=1))(document_representation)




In [66]:
# finally, add fc layers for classification
hidden = BatchNormalization()(document_representation)
hidden = Dense(20, activation='relu')(hidden)
hidden = Dropout(0.5)(hidden)

pred_sentiment = Dense(2, activation='softmax')(hidden)

model = Model(inputs=[document_input],
            outputs=[pred_sentiment])


In [67]:
optimizer = tf.keras.optimizers.SGD(momentum=0.9)
model.compile(loss=['categorical_crossentropy'],
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 15, 100)]    0                                            
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 15, 128)      1862980     input_6[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 15, 128)      98816       time_distributed_2[0][0]         
__________________________________________________________________________________________________
attention_layer_5 (AttentionLay (None, 15)           13000       bidirectional_5[0][0]            
____________________________________________________________________________________________

In [68]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# mc = ModelCheckpoint('/content/drive/MyDrive/Colab/sentiment/model/model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(X_train, Y_train, epochs=1, batch_size=64, validation_split=0.1)


ValueError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=2)
print('\n테스트 정확도:', test_acc)
        

52/52 - 7s - loss: 0.6929 - accuracy: 0.5137

테스트 정확도: 0.5137446522712708


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# loaded_model = load_model('/content/drive/MyDrive/Colab/sentiment/model/mode;.h5')

In [ ]:
def predict(new_sentence):

  new_sentence = preprocess_sentence(new_sentence)
  token = [word for word in new_sentence.split()]  
  tokenized_sentences = tokenizer.texts_to_sequences(token)
  tokenized_sentences = sequence.pad_sequences(tokenized_sentences, maxlen=max_len)
  pad_size = MAX_SENTENCES - tokenized_sentences.shape[0]

  if pad_size <= 0:
    tokenized_sentences = tokenized_sentences[:MAX_SENTENCES]
  else:
    tokenized_sentences = np.pad(
      tokenized_sentences, ((0, pad_size), (0, 0)),
      mode='constant', constant_values=0
    )

  pred_attention = model.predict(np.asarray([tokenized_sentences]))
  print(pred_attention)






In [ ]:
predict("	UGH I HATE QUEUEING")

[[0.47596094 0.52403903]]
